# Домашняяя работа-2019-06-05

## Задание1

In [1]:
import pandas as pd

In [15]:
df = pd.read_csv('./data-HW-6/keywords.csv')

In [16]:
df.shape

(100000, 2)

In [17]:
df.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

In [18]:
geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [19]:
test = 'москва звонят колокола'
test.split()

['москва', 'звонят', 'колокола']

In [20]:
def label_region(x):
    region = 'undefined'
    for w in x.split():
        for key, value in geo_data.items():
            if w in value:
                region = key
    return region

In [21]:
label_region(test)

'Центр'

In [22]:
df['region'] = df['keyword'].apply(label_region)

In [23]:
df.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [24]:
df[df['region'] != 'undefined'].head()

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
1236,банк санкт петербург,174375,Северо-Запад


## Задание 2

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
    - оценка 2 и меньше - низкий рейтинг
    - оценка 4 и меньше - средний рейтинг
    - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [33]:
ratings = pd.read_csv('./data-HW-6/ratings.csv')

In [34]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [37]:
def classify_ratings(x):
    rating = 'низкий рейтинг'
    if x > 2:
        rating = 'средний рейтинг'
    if x>4:
        rating = 'высокий рейтинг'
    return rating       

In [38]:
ratings['class'] = ratings['rating'].apply(classify_ratings)

In [39]:
ratings.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг


In [40]:
movies = pd.read_csv('./data-HW-6/movies.csv')

In [42]:
joined = ratings.merge(movies, on='movieId', how='left')
del joined['genres']
joined.head()

,userId,movieId,rating,timestamp,class,title
0,1,31,2.5,1260759144,средний рейтинг,Dangerous Minds (1995)
1,1,1029,3.0,1260759179,средний рейтинг,Dumbo (1941)
2,1,1061,3.0,1260759182,средний рейтинг,Sleepers (1996)
3,1,1129,2.0,1260759185,низкий рейтинг,Escape from New York (1981)
4,1,1172,4.0,1260759205,средний рейтинг,Cinema Paradiso (Nuovo cinema Paradiso) (1989)


## Задание 3

Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.

In [43]:
ratings.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг


In [46]:
cinema_fans = ratings.groupby('userId').count()

In [52]:
cinema_fans_IDs = list(cinema_fans[cinema_fans['movieId'] >= 100].index)

In [54]:
ratings_fans = ratings.loc[ratings['userId'].isin(cinema_fans_IDs)]

In [68]:
ratings_fans.head()

,userId,movieId,rating,timestamp,class
147,4,10,4.0,949810645,средний рейтинг
148,4,34,5.0,949919556,высокий рейтинг
149,4,112,5.0,949810582,высокий рейтинг
150,4,141,5.0,949919681,высокий рейтинг
151,4,153,4.0,949811346,средний рейтинг


In [69]:
ratings_fans_grouped = ratings_fans.groupby('userId').agg([min,max])

In [70]:
del ratings_fans_grouped['class']

In [71]:
ratings_fans_grouped.head()

movieId         rating        timestamp            
           min     max    min  max         min         max
userId                                                    
4           10    4006    1.0  5.0   949778714   949982274
5            3   48385    1.5  5.0  1163373044  1163375145
8           32   44004    0.5  5.0  1154389340  1154474527
15           1  161155    0.5  5.0   997937239  1469330735
17           6   34437    0.5  5.0  1127468587  1127476640

In [72]:
ratings_fans_grouped['diff'] = ratings_fans_grouped['timestamp']['max'] - ratings_fans_grouped['timestamp']['min']

In [73]:
ratings_fans_grouped.head()

movieId         rating        timestamp                   diff
           min     max    min  max         min         max           
userId                                                               
4           10    4006    1.0  5.0   949778714   949982274     203560
5            3   48385    1.5  5.0  1163373044  1163375145       2101
8           32   44004    0.5  5.0  1154389340  1154474527      85187
15           1  161155    0.5  5.0   997937239  1469330735  471393496
17           6   34437    0.5  5.0  1127468587  1127476640       8053

In [74]:
ratings_fans_grouped['diff'].mean() / 24 / 3600

455.2285713719898

## Задание 4

In [75]:
ratings = pd.read_csv('./data-HW-6/ratings.csv')

In [76]:
movies = pd.read_csv('./data-HW-6/movies.csv')

In [77]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [80]:
years = list(range(1950,2011))

In [111]:
x = 'Cinema Paradiso (Nuovo cinema Paradiso) (1989)'

In [112]:
import re
re.findall(r'\b\d+\b', x)[0]

'1989'

In [93]:
start = x.find('(')+1

In [94]:
end = x.find(')')

In [95]:
x[start:end]

'1995'

In [113]:
def find_year(x): 
    try:
        #start = x.find('(')+1
        #end = x.find(')')
        #year = x[start:end]
        year = x[-5:-1]
        if int(year) in years:
            year = int(year)
        else:
            year = 1900
    except:
        year = 1900
    return year

In [114]:
movies['year'] = movies['title'].apply(find_year)
movies.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [115]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()

,userId,movieId,rating,timestamp,title,genres,year
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,1995
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,1900
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,1996
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,1981
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,1989


In [127]:
import numpy as np
av_rating = joined.groupby('year').agg([np.mean])
del av_rating['movieId'], av_rating['userId'], av_rating['timestamp']

In [135]:
av_rating['rating'].sort_values('mean', ascending = False).head(10)

,mean
year,
1957,4.014241
1972,4.011136
1952,4.000000
1954,3.994220
1951,3.983539
1974,3.978704
1962,3.952446
1950,3.915254
1977,3.905786
